In [140]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers import Dropout ,LSTM ,Embedding ,Dense
from sklearn.model_selection import train_test_split

In [141]:
train = pd.read_csv('..//Twitter_sentiment_classification//train_E6oV3lV.csv')
test = pd.read_csv('..//Twitter_sentiment_classification//test_tweets_anuFYb8.csv')

In [142]:
train['tweet'] = train['tweet'].apply(lambda x : x.lower().strip())
test['tweet'] = test['tweet'].apply(lambda x : x.lower().strip())
train['tweet'] = train['tweet'].apply(lambda x : re.sub("[^a-zA-Z0-9\s]" ," ", x))
test['tweet'] = test['tweet'].apply(lambda x : re.sub("[^a-zA-Z0-9\s]" ," ", x))
train['tweet'][13]

' user  cnn calls  michigan middle school  build the wall  chant     tcot'

In [143]:
tokenize= Tokenizer(num_words=2000 , split=' ')
tokenize.fit_on_texts(train.tweet.values)
x = tokenize.texts_to_sequences(train.tweet.values)
x = pad_sequences(x)


In [144]:
x.shape

(31962, 39)

In [145]:
model = Sequential()

In [146]:
model.add(Embedding(2000 , 196))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(2 ,activation = "softmax"))
model.compile(loss = 'categorical_crossentropy' , optimizer = "adam" , metrics=['accuracy'])

In [147]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 196)         392000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               166400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 558,658
Trainable params: 558,658
Non-trainable params: 0
_________________________________________________________________


In [160]:
y = pd.get_dummies(train['label'])


In [161]:
x_train ,x_test ,y_train,y_test = train_test_split(x , y , test_size = 0.2 ,random_state =4)

In [162]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(25569, 39) (25569, 2)
(6393, 39) (6393, 2)


In [164]:
model.fit(x_train ,y_train ,batch_size=32 ,epochs=10,verbose=1)

Epoch 1/10
 1536/25569 [>.............................] - ETA: 1:02 - loss: 0.3069 - acc: 0.9258

KeyboardInterrupt: 

In [59]:
score ,acc = model.evaluate(x_test ,y_test)

6393/6393 [==============================] - 3s 451us/step


In [61]:
print("score" ,score)
print('acc' , acc)

score 0.3112635965028485
acc 0.9408728296574378


In [103]:
list1 = []
list2 = []
for twt in test.tweet:
    p = twt
    twt = tokenize.texts_to_sequences(twt)
    twt = pad_sequences(twt, maxlen=39, dtype='int32', value=0)
    sentiment = model.predict(twt,batch_size=1,verbose = 2)
    if(np.argmax(sentiment) == 0):
        list1.append((p , 'neagtive'))
    elif (np.argmax(sentiment) == 1):
        list2.append((p , 'positive'))
    


KeyboardInterrupt: 

## using Logistic regression classifier

In [168]:
from sklearn.linear_model import LogisticRegression

In [169]:
lr = LogisticRegression()

In [171]:
y1 = train['label']


In [172]:
x_train ,x_test ,y_train,y_test = train_test_split(x , y1 , test_size = 0.2 ,random_state =4)

In [173]:
lr.fit(x_train ,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [174]:
lr.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [175]:
import sklearn.metrics  as mt
mt.accuracy_score(lr.predict(x_test) , y_test)

0.9285155638980135

## using randomforest

In [176]:
from sklearn.ensemble import RandomForestClassifier

In [182]:
rf = RandomForestClassifier(max_depth =15)

In [183]:
rf.fit(x_train ,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [184]:
rf.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [185]:
import sklearn.metrics as mt
print(mt.accuracy_score(rf.predict(x_test) , y_test))

0.9380572501173158
